<a href="https://colab.research.google.com/github/iamsusiep/slp2019/blob/master/Praat_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!apt -qq install -y sox

sox is already the newest version (14.4.2-3ubuntu0.18.04.1).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.


In [0]:
!pip install sox

In [0]:
!pip install praat-parselmouth
!pip install config

     |████████████████████████████████| 9.0MB 4.8MB/s 
  Created wheel for config: filename=config-0.4.2-cp36-none-any.whl size=15134 sha256=b6a43441666569817df0731d75cf3a7c52ec0774a5fa369a26c4aa4c57d60d2e
  Stored in directory: /root/.cache/pip/wheels/51/7d/db/0e38d2ec57843d00cc39f8df3686984ccec689694f7bc78a38
Successfully built config


In [0]:
import os
import pandas as pd
import parselmouth
from parselmouth.praat import call  # pylint: disable=no-name-in-module, import-error
from config import Config
import glob
from sklearn.preprocessing import minmax_scale
import numpy as np
import sys
import glob


def extract_features(filenames):

    output_df = pd.DataFrame(columns=['filename', 
                                    'Min Pitch', 'Max Pitch', 'Mean Pitch', 'Sd Pitch', 
                                    'Min Intensity', 'Max Intensity', 'Mean Intensity', 'Sd Intensity', 
                                    'Jitter', 'Shimmer', 'HNR', 'Energy'])
    ipu_df = pd.DataFrame(columns=['filename','start_time', 'end_time'])
    count = 0
    for fn in filenames:
        count += 1
        if count % 100 == 0:
            print('completed:', count)
        """
        segment to IPU
        """
        try:
            sound = parselmouth.Sound(fn)
            pitch = call(sound, 'To Pitch', 0.0, 75.0, 600.0)
            intensity = call(sound, 'To Intensity', 75.0, 0.0, True)
            point_process = call(sound, 'To PointProcess (periodic, cc)', 75.0, 600.0)
            harmonicity = call(sound, 'To Harmonicity (cc)', 0.01, 75, 0.1, 1.0)

            silence = call(sound, "To TextGrid (silences)",100, 0.0, -25.0, 0.05, 0.1, "silent", "sounding")
            num_intervals = call(silence, "Get number of intervals",1)
            start_times = []
            end_times = []
            for i in range(1, num_intervals+1):
                label = call(silence, "Get label of interval", 1, i)
                if label == "sounding":
                    start_times.append(call(silence, "Get start time of interval",1,  i))
                    end_times.append(call(silence, "Get end time of interval", 1, i))
            
            ipu_df.append(pd.Series([os.path.basename(fn), start_times,end_times],
                                                    index=ipu_df.columns), ignore_index=True)

            min_pitches = []
            max_pitches = []
            mean_pitches = []
            sd_pitches = []

            min_intensities = []
            max_intensities = []
            mean_intensities = []
            sd_intensities = []

            jitter_list = []
            shimmer_list = []
            hnr_list = []
            energy_list = []        

            for t1, t2 in zip(start_times, end_times):
                # pitch
                min_pitch = call(pitch, 'Get minimum', t1, t2, 'Hertz', 'Parabolic')
                max_pitch = call(pitch, 'Get maximum', t1, t2, 'Hertz', 'Parabolic')
                mean_pitch = call(pitch, 'Get mean',t1, t2, 'Hertz')
                sd_pitch = call(pitch, 'Get standard deviation', t1, t2, 'Hertz')
    
                min_pitches.append(min_pitch)
                max_pitches.append(max_pitch)
                mean_pitches.append(mean_pitch)
                sd_pitches.append(sd_pitch)

                # intensity
                min_intensity = call(intensity, 'Get minimum', t1, t2, 'Parabolic')
                max_intensity = call(intensity, 'Get maximum', t1, t2, 'Parabolic')
                mean_intensity = call(intensity, 'Get mean', t1, t2, 'energy')
                sd_intensity = call(intensity, 'Get standard deviation', t1, t2)
    
                min_intensities.append(min_intensity)
                max_intensities.append(max_intensity)
                mean_intensities.append(mean_intensity)
                sd_intensities.append(sd_intensity)
    
                # jitter, shimmer
                jitter = call(point_process, 'Get jitter (local)', t1, t2, 0.0001, 0.02, 1.3)
                shimmer = call([sound, point_process], 'Get shimmer (local)', t1, t2, 0.0001, 0.02, 1.3, 1.6)
                jitter_list.append(jitter)
                shimmer_list.append(shimmer)

                # HNR
                hnr = call(harmonicity, "Get mean", t1, t2)
                hnr_list.append(hnr)
    
                # Energy
                energy = call(sound, 'Get energy', t1, t2)
                energy_list.append(energy)
            min_pitches = np.array(min_pitches)
            max_pitches = np.array(max_pitches)
            mean_pitches = np.array(mean_pitches)
            sd_pitches = np.array(sd_pitches)

            min_intensities = np.array(min_intensities)
            max_intensities = np.array(max_intensities)
            mean_intensities = np.array(mean_intensities)
            sd_intensities = np.array(sd_intensities)

            jitter_list = np.array(jitter_list)
            shimmer_list = np.array(shimmer_list)
            hnr_list = np.array(hnr_list)
            energy_list = np.array(energy_list) 

            min_pitches = min_pitches[~np.isnan(min_pitches)]
            max_pitches = max_pitches[~np.isnan(max_pitches)]
            mean_pitches = mean_pitches[~np.isnan(mean_pitches)]
            sd_pitches = sd_pitches[~np.isnan(sd_pitches)]

            min_intensities = min_intensities[~np.isnan(min_intensities)]
            max_intensities = max_intensities[~np.isnan(max_intensities)]
            mean_intensities = mean_intensities[~np.isnan(mean_intensities)]
            sd_intensities = sd_intensities[~np.isnan(sd_intensities)]

            jitter_list = jitter_list[~np.isnan(jitter_list)]
            shimmer_list = shimmer_list[~np.isnan(shimmer_list)]
            hnr_list = hnr_list[~np.isnan(hnr_list)]
            energy_list = energy_list[~np.isnan(energy_list)]

            min_pitch = sum(min_pitches / np.linalg.norm(min_pitches))/len(min_pitches)
            max_pitch = sum(max_pitches / np.linalg.norm(max_pitches))/len(max_pitches)
            mean_pitch = sum(mean_pitches / np.linalg.norm(mean_pitches))/len(mean_pitches)
            sd_pitch = sum(sd_pitches / np.linalg.norm(sd_pitches))/len(sd_pitches)
        
            min_intensity = sum(min_intensities / np.linalg.norm(min_intensities))/len(min_intensities)
            max_intensity = sum(max_intensities / np.linalg.norm(max_intensities))/len(max_intensities)
            mean_intensity = sum(mean_intensities / np.linalg.norm(mean_intensities))/len(mean_intensities)
            sd_intensity = sum(sd_intensities / np.linalg.norm(sd_intensities))/len(sd_intensities)

            jitter = sum(jitter_list / np.linalg.norm(jitter_list))/len(jitter_list)
            shimmer = sum(shimmer_list / np.linalg.norm(shimmer_list))/len(shimmer_list)
            hnr = sum(hnr_list / np.linalg.norm(hnr_list))/len(hnr_list)
            energy = sum(energy_list / np.linalg.norm(energy_list))/len(energy_list)
    
            output_df = output_df.append(pd.Series([os.path.basename(fn), min_pitch, max_pitch, mean_pitch, sd_pitch,
                                                    min_intensity, max_intensity, mean_intensity, sd_intensity,
                                                    jitter, shimmer, hnr, energy],
                                                    index=output_df.columns), ignore_index=True)
        except:
            print(sys.exc_info())
            continue
    output_df.to_csv("drive/My Drive/praat5.csv", index=None, header=True)
    ipu_df.to_csv("drive/My Drive/ipu5.csv", index=None, header=True)
    return output_df, ipu_df
myargs = glob.glob("drive/My Drive/align_speech/*.wav")
df, ipu_df = extract_features(myargs)

(<class 'parselmouth.PraatError'>, PraatError('Sound "untitled": shorter than window length.\nThe physical duration of the sound (the number of samples times the sampling period) in an intensity analysis should be at least 6.4 divided by the minimum pitch (75 Hz), i.e. at least 0.08533333333333334 s, instead of 0.08 s.\nSound "untitled": intensity analysis not performed.\n',), <traceback object at 0x7f0d7b7c5ac8>)
completed: 100
completed: 200
completed: 300
completed: 400
(<class 'ZeroDivisionError'>, ZeroDivisionError('division by zero',), <traceback object at 0x7f0d7b7d3508>)
completed: 500
completed: 600
(<class 'ZeroDivisionError'>, ZeroDivisionError('division by zero',), <traceback object at 0x7f0d7b519d48>)
completed: 700
completed: 800
completed: 900
(<class 'ZeroDivisionError'>, ZeroDivisionError('division by zero',), <traceback object at 0x7f0d7b7d4e88>)
completed: 1000
(<class 'ZeroDivisionError'>, ZeroDivisionError('division by zero',), <traceback object at 0x7f0d7b7d22c8>)